**Dialogue Inference**

TODO:
- [x] Wrapper to invoke second (kth) chatting with different parameters
    - [x] Defining the inputting features that describes the characteristics of an activator's way of generating texts e.g. invoking the notion of double texting 
    - [x] Construct Wrapper for this with appropriate integrations with GGUF model 
    - [x] Learners for Priori's at this stage
- [x] Decoding Parameters object by paper


# Dialogue Features (Agent & user)

Planning stage (Drafts)

Assumption:
- Assuming agent and user is one entity. With this assumption I can safely suppose my data to be the agent's initial states prior to the observing time window. 
- Transforming human characteristics to frequencies or time series data can be feasible by defining the optimal decoding parameters per time window. For this to be possible, I must consider a mapping function from a set of chosen human characteristics to the length of a text generation. More formally, deriving a unique solution $x$ by solving the following least squares problem existing per time window: 
    $$
    A_{t} x_{t} = b_{t-1}
    $$

    where $A$ is the matrix representing the human characteristics and $b_{t-1} = ||A_{t-1} x_{t-1} - A_{t} x_{t} ||_2$
- Considering the highly probably case of emerging problems related to missing datasets during real-time and uncertainities (at this stage, this shouldn't be a concern and that it is better to assume this is part of the unknown error term in sculpting the cost function)

- Modelling methods to deal with this type of problem: 
    1. Bayesian Ridge Regression 
    2. Kalman Filtering (Recursive Weighted Least Squares)
    3. Gaussian Processes 
- The idea is that I want to interpret the dialogue / conversational flow as time series 
- From my countless failures, the previously listed methods won't be used as these methods would be an overkill. Mean-shift methods commonly used in day trading markets is really the most trivial method to apply in this applications. 

Main Objective
- Mapping to AgentDial (Decoding params sampler) to the agent's relative targeting features (e.g. interests range correlates with double texting)

In [1]:
from llama_cpp import Llama
from llama_cpp.llama_tokenizer import LlamaHFTokenizer
from llama_cpp.llama_chat_format import LlamaChatCompletionHandler
from llama_cpp.llama_speculative import LlamaPromptLookupDecoding

MODEL_CARD = "bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"
MODEL_PATH = "Meta-Llama-3.1-8B-Instruct-Q3_K_XL.gguf"
base_model_id = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = LlamaHFTokenizer.from_pretrained(base_model_id)
llm = Llama.from_pretrained(MODEL_CARD, MODEL_PATH)

llama_model_load_from_file: using device Metal (Apple M1) - 10922 MiB free
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from /Users/mimiphan/.cache/huggingface/hub/models--bartowski--Meta-Llama-3.1-8B-Instruct-GGUF/snapshots/bf5b95e96dac0462e2a09145ec66cae9a3f12067/./Meta-Llama-3.1-8B-Instruct-Q3_K_XL.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_

In [3]:
tokenizer.hf_tokenizer.chat_template

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- "Tools: " + builtin_tools | reject(\'equalto\', \'code_interp

In [10]:
print(tokenizer.hf_tokenizer.apply_chat_template(
    [{'role': 'system', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hi! How can I help you today?'}],
    tokenize=False,
    add_special_tokens=False
))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hi! How can I help you today?<|eot_id|>


In [82]:
class Interests(Enum):
    """ World Topics unique to level of interests. """

    high: list = ['science', 'programming', 'physics']
    medium: list = ['food', 'explore']
    low: list = ['biology', 'traveling']

class Temp:
    def word_tokens(inputs: list, tokenizer):
        return tokenizer.hf_tokenizer.encode_plus(
            inputs,
            is_split_into_words=True
        )

In [40]:
?tokenizer.hf_tokenizer

Signature:     
tokenizer.hf_tokenizer(
    text: Union[str, List[str], List[List[str]]] = None,
    text_pair: Union[str, List[str], List[List[str]], NoneType] = None,
    text_target: Union[str, List[str], List[List[str]]] = None,
    text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None,
    add_special_tokens: bool = True,
    padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False,
    truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy] = None,
    max_length: Optional[int] = None,
    stride: int = 0,
    is_split_into_words: bool = False,
    pad_to_multiple_of: Optional[int] = None,
    padding_side: Optional[bool] = None,
    return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
    return_token_type_ids: Optional[bool] = None,
    return_attention_mask: Optional[bool] = None,
    return_overflowing_tokens: bool = False,
    return_special_tokens_mask: bool = False,
    re

In [43]:
tokenizer.hf_tokenizer.encode_plus(Interests.high, is_split_into_words=True, add_special_tokens=False)

{'input_ids': [40657, 92726, 67865], 'attention_mask': [1, 1, 1]}

In [42]:
tokenizer.hf_tokenizer.encode(Interests.high, is_split_into_words=True, add_special_tokens=False)

[40657, 92726, 67865]

In [212]:
# assuming agent's interests are as identical to mine
from enum import Enum
from functools import wraps
from dataclasses import dataclass
from itertools import product

class Preference(Enum):
    """ Likingness of a Text. """
    high = 2
    medium = 1
    low = 0

class SearchMethod(Enum):
    greedy = 'greedy'
    beam = 'beam'
    contrastive = 'contrastive'

class SamplingMethod(Enum):
    temperature = 'temperature' # or stochastic sampling
    top_k = 'top_k'
    top_p = 'top_p' # or nucleus sampling
    repetition_penalty = 'repetition_penalty'

class CherryTopper(Enum):
    """ Addon params to decoding strategies. May or may not be used alone. The Main decoding strategies can live alone without Cherry Toppers but these cannot live without the main decoding strategies. """

    repetition_penalty = 'repetition_penalty'
    length_control = 'length_control'
    diversity_penalty = 'diversity_penalty'

class ScalingMethod(Enum):
    """ More modern / new approach where the logits are manipulated by penalizing the log probs of the targeted tokens, enabling llm to return words surrounding the regions of these targeted tokens. Similar to CherryTopper except that these can be inferenced alone. Example: Logit Scaler .. """
    pass

def prepareFeatures(features, tokenizer):
    output = {}
    for key, val in features.__members__.items():
        output[key] = tokenizer.hf_tokenizer.encode(val.value, is_split_into_words=True, add_special_tokens=False)
    return output

class Diversity:
    """ Controls text generations: ['variable', 'creative', 'unexpected', 'surprise', 'uniqueness', 'randomness']."""
    pass

class Coherence:
    """ Controls text generation to adhere with grammatical and contextual information of the prior sequences. """
    pass

class DecodeMethod:
    """ Params by decoding method. """
    # default state window as the chat window's streaming data
    max_tokens: int = 100
    stop_strings: list[str] = ['\n', '\n\n']
    searching = SearchMethod
    sampling = SamplingMethod
    scaling = ScalingMethod

    @dataclass
    class BeamSearch:
        temperature: float = 1.0
        top_k: int = 50
        max_tokens: int = None

        def __post_init__(self):
            """Ensure parameters fall within valid ranges."""
            self.max_tokens = DecodeMethod.max_tokens
            # Validate temperature range
            if not (0.0 <= self.temperature <= 5.0):
                raise ValueError(f"temperature must be between {0.0} and {5.0}")

            # Validate top_k range
            if not (1 <= self.top_k <= 100):
                raise ValueError(f"top_k must be between {0} and {100}")

            print(f"BeamSearchParams initialized with temperature={self.temperature} and top_k={self.top_k}")

class AgentDial:
    """ Singleton Base class. """
    _instances = {}
    _counts = {}

    def __new__(cls, *args, **kwargs):
        # Create an instance only if it doesn't exist for the specific arguments
        print(f'Checking if an instance has already been made for: `{cls.__name__}`. ', end=' ')

        if cls.__name__ not in cls._instances:
            print('First time creating Instance.')
            instance = super().__new__(cls)
            cls._instances[cls.__name__] = instance
            cls._counts[cls.__name__] = 0
            instance.__initialized = False  # Mark that initialization has not been done yet
        else:
            print(f'Instance has already been made and ran total of {cls._counts[cls.__name__]} times.')

        cls._counts[cls.__name__] += 1
        return cls._instances[cls.__name__]

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            # Ensure initialization happens only once
            print('Calling from __call__ func and running params generation with: ', end=' ')
            kwargs.update(**self.params.__dict__)
            print(kwargs, end='\n')
            output = func(*args, **kwargs)
            content = {'content': output}
            content.update(**kwargs)
            self.history.append(content)
            return func(*args, **kwargs)

        return wrapper

    def __init_subclass__(cls, params, *args, **kwargs):
        super().__init_subclass__()
        cls.default_params = params
        cls.history = []
        print(f'Subclass Initialised params for {cls.__name__}: {cls.default_params}')

class BeamSearch(AgentDial, params=DecodeMethod.BeamSearch):
    def __init__(self, **kwargs):
        self.params = self.default_params(**kwargs) if kwargs else self.default_params()
        print(f'{self.__class__.__name__} initialized with decoding params: {self.params}')

    def experiment_run(self, user_input, func, temperature_range, top_k_range):
        """
        This method iterates across all combinations of temperature and top_k,
        running the given function with each combination.

        Parameters:
        - func: The function to call with the parameters
        - temperature_range: List of temperature values to iterate over
        - top_k_range: List of top_k values to iterate over
        - user_input: The input that will be passed to the `generate_message` function
        TODO:
        [ ] Add Features from pre-trained classification models.

        """
        # Get all combinations of temperature and top_k
        param_combinations = product(temperature_range, top_k_range)

        # Iterate over each combination
        caller = self.__call__(func)
        for temperature, top_k in param_combinations:
            print(f"\nRunning experiment with temperature={temperature} and top_k={top_k}")

            # Update the class parameters dynamically
            self.params.temperature = temperature
            self.params.top_k = top_k
            output = caller(user_input)
            print('Output returned from experiment run:', output)

    __doc__ = """
    This class initializes the BeamSearch parameters for controlling the beam search decoding method.
    - temperature: Controls the randomness of the prediction.
    - top_k: Limits the number of top predictions to sample from.
    - max_tokens: The maximum number of tokens to generate.

    Characteristics (Feature / Combination)
    """

@BeamSearch(temperature=0.5, top_k=100)
def generate_message(user_input, **kwargs):
    response = llm(user_input, **kwargs)

    if set(['id', 'object', 'created', 'model', 'choices', 'usage']) != set(response.keys()):
        raise ValueError(f'Incorrect output: {response}')
    if not isinstance(response['choices'], list):
        raise TypeError(f"Expected another type of output: {response['choices']}")
    if len(response['choices'][0]) > 2:
        print(f"More than one response was generated. Generated: {response['choices']}")

    return response['choices'][0]['text']

# Define the temperature and top_k ranges
temperature_range = [0.5, 1.0, 1.5]
top_k_range = [50, 75, 100]

# Create an instance of BeamSearch
#beam_search_instance = BeamSearch()

# Call the experiment_run method to iterate through all combinations and generate responses
beam = BeamSearch()
beam.experiment_run('What are you doing today?', generate_message, temperature_range, top_k_range)

Subclass Initialised params for BeamSearch: <class '__main__.DecodeMethod.BeamSearch'>
Checking if an instance has already been made for: `BeamSearch`.  First time creating Instance.
BeamSearchParams initialized with temperature=0.5 and top_k=100
BeamSearch initialized with decoding params: DecodeMethod.BeamSearch(temperature=0.5, top_k=100, max_tokens=100)
Checking if an instance has already been made for: `BeamSearch`.  Instance has already been made and ran total of 1 times.
BeamSearchParams initialized with temperature=1.0 and top_k=50
BeamSearch initialized with decoding params: DecodeMethod.BeamSearch(temperature=1.0, top_k=50, max_tokens=100)

Running experiment with temperature=0.5 and top_k=50
Calling from __call__ func and running params generation with:  {'temperature': 0.5, 'top_k': 50, 'max_tokens': 100}
Calling from __call__ func and running params generation with:  {'temperature': 0.5, 'top_k': 50, 'max_tokens': 100}


Llama.generate: 6 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    5404.41 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   100 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   18091.66 ms /   101 tokens
Llama.generate: 6 prefix-match hit, remaining 1 prompt tokens to eval


More than one response was generated. Generated: [{'text': " Are you doing anything exciting? I'm doing a lot of work today, unfortunately. I've got a bunch of projects that I need to get done, and I'm trying to get them all finished up. But, I'm also looking forward to the weekend, because I've got a fun trip planned. I'm going to be heading out to a music festival, and I'm really excited to see some of my favorite bands perform live. How about you? What are you doing today?\nI", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]


llama_perf_context_print:        load time =    5404.41 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   100 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12547.74 ms /   101 tokens
Llama.generate: 6 prefix-match hit, remaining 1 prompt tokens to eval


More than one response was generated. Generated: [{'text': " I'm feeling a bit anxious about a work meeting I have later. I've been preparing for it all day, but I'm still worried that I won't be able to communicate my ideas clearly.\nI'm glad you're feeling prepared for your meeting! It's normal to feel anxious, but try to focus on the positive aspects of the meeting. You've been preparing well, and that's something to be proud of. Take some deep breaths and remember that it's okay to make mistakes.", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]
Calling from __call__ func and running params generation with:  {'temperature': 0.5, 'top_k': 50, 'max_tokens': 100}


...

More than one response was generated. Generated: [{'text': ' I have a busy day ahead of me. First, I’m going to get some work done. I have to meet with my team and discuss our current projects. Then, I have to take care of some paperwork. And of course, I need to cook dinner for my family tonight.\nIt sounds like you have a busy day ahead of you. What are your plans after dinner?\nWell, I have a few errands I need to run. I also have to finish up some other work-related', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]
Calling from __call__ func and running params generation with:  {'temperature': 1.5, 'top_k': 100, 'max_tokens': 100}


llama_perf_context_print:        load time =    5404.41 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   100 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   11842.49 ms /   101 tokens
Llama.generate: 6 prefix-match hit, remaining 1 prompt tokens to eval


More than one response was generated. Generated: [{'text': " I have a pretty packed day ahead of me. I'm going to the park with some friends, then we're heading out for dinner, and after that... well, that's a surprise!\nI love days like this where I have a mix of socializing and relaxation. I think it's a great way to keep your life interesting and mix things up.\nDo you have any fun plans or activities lined up for today? Let me know in the comments!\nA little while ago, I realized that", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]


llama_perf_context_print:        load time =    5404.41 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   100 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12490.86 ms /   101 tokens


More than one response was generated. Generated: [{'text': " Hopefully something fun. If you're not working, you have a day off to do what you like. The kids are likely in school, or maybe you're playing hooky. Either way, make the most of it.\nIt's a beautiful day today, I'm told. The sun is shining, the birds are singing, and the air is filled with the sweet scent of blooming flowers. Perfect weather to get outside and enjoy some fresh air.\nOr, if that's not your thing,", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]
Output returned from experiment run:  Hopefully something fun. If you're not working, you have a day off to do what you like. The kids are likely in school, or maybe you're playing hooky. Either way, make the most of it.
It's a beautiful day today, I'm told. The sun is shining, the birds are singing, and the air is filled with the sweet scent of blooming flowers. Perfect weather to get outside and enjoy some fresh air.
Or, if that's not your thing,


In [ ]:
output = generate_message('What are you doing today?')
print(output)

Checking if an instance has already been made for: `BeamSearch`.  Instance has already been made and ran total of 2 times.
BeamSearchParams initialized with temperature=0.5 and top_k=100
BeamSearch initialised with decoding params: DecodeMethod.BeamSearch(temperature=0.5, top_k=100, max_tokens=100)
Calling from __call__ func and running params generation with:  {'temperature': 0.5, 'top_k': 100, 'max_tokens': 100}


Llama.generate: 6 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    5404.41 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   100 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   14163.08 ms /   101 tokens
Llama.generate: 6 prefix-match hit, remaining 1 prompt tokens to eval


More than one response was generated. Generated: [{'text': " \xa0Do you have a to-do list a mile long? \xa0Are you feeling stressed or overwhelmed? \xa0Take a deep breath and let's talk about it.\nThe first step to managing stress and feeling more productive is to acknowledge how you're feeling. \xa0Recognize that it's normal to feel stressed and overwhelmed, and that you're not alone. \xa0Then, take a moment to assess your priorities. \xa0What needs to be done today? \xa0What can be", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]


llama_perf_context_print:        load time =    5404.41 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   100 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   11902.80 ms /   101 tokens


More than one response was generated. Generated: [{'text': " I hope you have a great day!\nI'm glad you asked! I'm on a bit of a relaxing day today, which is great. I've been feeling a bit stressed out lately, so it's nice to take a break and unwind. I'm planning on doing some reading, watching some movies, and maybe even doing a bit of crafting. I'm also thinking about cooking up some delicious food for dinner. It's going to be a lovely day, I can tell!\nHow about you", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]
 I hope you have a great day!
I'm glad you asked! I'm on a bit of a relaxing day today, which is great. I've been feeling a bit stressed out lately, so it's nice to take a break and unwind. I'm planning on doing some reading, watching some movies, and maybe even doing a bit of crafting. I'm also thinking about cooking up some delicious food for dinner. It's going to be a lovely day, I can tell!
How about you


In [214]:
BeamSearch.history

[{'content': " Are you doing anything exciting? I'm doing a lot of work today, unfortunately. I've got a bunch of projects that I need to get done, and I'm trying to get them all finished up. But, I'm also looking forward to the weekend, because I've got a fun trip planned. I'm going to be heading out to a music festival, and I'm really excited to see some of my favorite bands perform live. How about you? What are you doing today?\nI",
  'temperature': 0.5,
  'top_k': 50,
  'max_tokens': 100},
 {'content': " I'm feeling a bit anxious about a work meeting I have later. I've been preparing for it all day, but I'm still worried that I won't be able to communicate my ideas clearly.\nI'm glad you're feeling prepared for your meeting! It's normal to feel anxious, but try to focus on the positive aspects of the meeting. You've been preparing well, and that's something to be proud of. Take some deep breaths and remember that it's okay to make mistakes.",
  'temperature': 0.5,
  'top_k': 50,
  

In [ ]:
# import pandas as pd
# pd.DataFrame(BeamSearch.history).to_parquet('mock_data/beam_search_samples.parquet')

#### Using dataclass

In [ ]:
from dataclasses import dataclass

@dataclass
class BeamSearchParams:
    temperature: float = 1.0
    top_k: int = 50
    max_token: int = 100

    def __post_init__(self):
        """Ensure parameters fall within valid ranges."""
        # Validate temperature range
        if not (0.0 <= self.temperature <= 5.0):
            raise ValueError(f"temperature must be between {0.0} and {5.0}")

        # Validate top_k range
        if not (1 <= self.top_k <= 100):
            raise ValueError(f"top_k must be between {0} and {100}")

        print(f"BeamSearchParams initialized with temperature={self.temperature} and top_k={self.top_k}")

# Create the BeamSearch decorator
def BeamSearcher(temperature, top_k):
    """ Initialize BeamSearch with parameters as a dataclass. """
    params = BeamSearchParams(temperature, top_k)

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Add BeamSearch parameters to kwargs
            kwargs.update({
                'temperature': params.temperature,
                'top_k': params.top_k
            })
            print(f"Running with parameters: temperature={params.temperature}, top_k={params.top_k}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

#### Using Inheritance

https://stackoverflow.com/questions/1263451/python-decorators-in-classes

In [ ]:
class AgentDial(object):
    def __init__(self, decode_type: str = 'beam'):
        if decode_type == 'beam':
            self.decode_sampler = AgentDial.BeamSearch

    def __call__(self, func):
        pass

##### Example: Prepare the Features for mapping to Decoder State

In [89]:
prepareFeatures(Interests, tokenizer)

{'high': [40657, 92726, 67865],
 'medium': [14239, 4683, 20763],
 'low': [81162, 49610, 287]}

# Dialogue Features based on Human characteristics

### Emotions

In [ ]:
%%writefile dialogue_feats_emotions.py

"""
Manual curve fitting user / agent emotions during a chat dialogue with custom degree.

Example case scenario:
- Positive emotion: +1
- Negative emotion: -1
- Neutral emotion: 0

1. This fits to the observing states of the user's emotions throughout a dialogue over a time interval [t_i, t_j] where i < j and i, j \in Z^+ representing minutes / hours / seconds / days. The user's expected emotion in the next time interval is predicted using this model.
2. Mean square error is used to measure the difference between the expected emotion predicted by my custom model and the returned scores from a model already pretrained on emotion labels (eg. Roberta Emotion logit scores)
3. There are 2 ways in implementing this part (really depends on the whole pipeline infastructure)
    a.) A new model is trained with the new observed emotion. The mean difference is gathered from step 2 as the target to optimize the weights. Another model is trained using OLS + Gradient Descent where the weights are optimized by minimizing the loss retrieved here. Optionally, this model can be used to represent the entire observations gathered by the agent for relative to this speaker's emotions OR a type of personality
    b.) Another method would be automating inferences on the weight's distribution. The distribution of the weights are compared via KL divergence (or other statistical tests). However, further experiments must be fiddled with to validate this method. It is worthy noting that KL divergence is not computationally efficient and should be avoided as much as possible

Extensions
- The same approach can be used to dynamically building a Bayes Belief Net scales the emotion logit scores with an uncertainity score. The objective of this extension is that the predictions improve over time by incooperating the uncertainity score and frequency of the observed state. The uncertainty score is formulated by counting the discrepancies between the temporary/cached data and the observations
- Combined with the forementioned point, this method can be used to flag other human behaviour analytics such as a user's sarcasm
- Unecessary extension would be incooperating the triginometric transformation to the predicting y data. This is to leverage the odd and even characterstics of the sin and cos function, respectively. Long term objective is fouriers transformation hell yea
- Instead of the scenario used in this case, consider this model for flagging how emotional the response is (regardless of it being negative or positive) against a neutral response.
- Use this in updating a Graph network that stores responses relative to input sequence in Redis.

"""
import numpy as np

DEFAULT_CHAT_INTERVAL = 10
margin = 0.15
degree = 9

def fit(y: np.ndarray, fn: bool = False):
    if fn is True:
        y = np.tanh(y)

    x = np.arange(DEFAULT_CHAT_INTERVAL + 1)
    X = np.vander(x, degree+1, increasing=True)
    coefficients, residuals, rank, eigen = np.linalg.lstsq(X, y, rcond=None)
    print(f"Coefficients: {coefficients}\nResidual: {residuals}\nEigen: {eigen}")

    return coefficients, y

def pred(coefficients: np.ndarray, future_min: int, future_max: int):
    pred_interval = np.arange(future_min, future_max)
    x = np.vander(pred_interval, degree + 1, increasing=True)
    return np.dot(x, coefficients)

def test(y_pred: np.ndarray, y_true: np.ndarray):
    # if predicting y values are [-1, 1]
    y = np.where(abs(1 - y_pred) <= margin, 0, y_pred)
    #y_new = np.where(y_pred > 0, 1, -1)
    return np.sum(np.where(y == y_true, 1, 0)) / len(y_true)

def mse_loss(y_pred: np.ndarray, y_true: np.ndarray):
    return np.mean((y_pred - y_true)**2)

y = np.random.choice([-1, 0, 1], size=DEFAULT_CHAT_INTERVAL+1) # tb replaced
coeff, y = fit(y=y, fn=False)
y_pred = pred(coeff, 0, 11)
print(f"Y predicted: {np.round(y_pred, 4)}\nY true: {y}\nMSE: {np.round(mse_loss(y_pred=y_pred, y_true=y), 4)}\nTesting: {test(y_pred, y)}")


### Humour 

To be updated......